### Data Reading

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@databricksetenimit.dfs.core.windows.net/orders")

In [0]:
display(df)

In [0]:
df = df.withColumnRenamed('_rescued_data', 'rescued_data')
display(df)

In [0]:
df = df.drop("rescued_data")
display(df)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [0]:
df = df.withColumn(
    "order_date",
    to_timestamp(col("order_date"))
)
display(df)

In [0]:
df = df.withColumn("year", year(col("order_date")))
display(df)

In [0]:
df1 = df1.withColumn("rankflag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

In [0]:
df1 = df1.withColumn("rowflag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

### Classes-OOP

In [0]:
class windows():

    def dense_rank(self,df):
        df_dense_rank = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank
    
    def rank(self,df):
        df_rank = df.withColumn("rankflag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_rank
    
    def row_number(self,df):
        df_row_number = df.withColumn("rowflag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_row_number
    

In [0]:
df_new = df
df_new.display()


In [0]:
Windows = windows()
df_new = Windows.dense_rank(df_new)
display(df_new)


### Data Writing

In [0]:
df.write.format("delta").mode("append").save("abfss://silver@databricksetenimit.dfs.core.windows.net/orders")